Reference to : https://medium.com/@yulongtsai/https-medium-com-yulongtsai-titanic-top3-8e64741cc11f


In [ ]:
import numpy as np 
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
plt.style.use('fivethirtyeight')
import warnings
warnings.filterwarnings('ignore')
%matplotlib inline